In [5]:
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
import os
import tqdm
import numpy as np
import tensorflow as tf
from scipy import signal
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
# Find data files.

data_folder = "./data/dataset/data/training_data/"

# Find the patient data files.
patient_files = find_patient_files(data_folder)
num_patient_files = len(patient_files)

if num_patient_files==0:
    raise Exception('No data was provided.')

# Create a folder for the model if it does not already exist.
#os.makedirs(model_folder, exist_ok=True)
#TODO: remove this:
#classes = ['Present', 'Unknown', 'Absent']
#num_classes = len(classes)

murmur_classes = ['Present', 'Unknown', 'Absent']
num_murmur_classes = len(murmur_classes)
outcome_classes = ['Abnormal', 'Normal']
num_outcome_classes = len(outcome_classes)

# Extract the features and labels.
#if verbose >= 1:
#    print('Extracting features and labels from the Challenge data...')

data = []
murmurs = list()
outcomes = list()
NEW_FREQUENCY = 500

for i in tqdm.tqdm(range(num_patient_files)):
    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files[i])
    current_recordings, freq = load_recordings(data_folder, current_patient_data, get_frequencies=True)
    for j in range(len(current_recordings)):
        data.append(signal.resample(current_recordings[j], int((len(current_recordings[j])/freq[j]) * NEW_FREQUENCY)))



 39%|███▉      | 371/942 [00:15<00:24, 23.37it/s]


KeyboardInterrupt: 

In [ ]:
    current_murmur = np.zeros(num_murmur_classes, dtype=int)
    murmur = get_murmur(current_patient_data)
    if murmur in murmur_classes:
        j = murmur_classes.index(murmur)
        current_murmur[j] = 1
    murmurs.append(current_murmur)